# 12. 자바 예외처리 + 표준 입출력


## 예외처리

- 컴파일 오류 : 프로그램 코드 작성 중 발생하는 문법적 오류
- 실행 오류 : 실행 중인 프로그램이 의도치않은 동작을 하거나 프로그램이 중지되는 오류
- 자바는 예외처리를 통하여 프로그램의 비정상 종료를 막고 log를 남길 수 있음

### 오류와 예외 클래스
- 시스템 오류 : 가상 머신에서 발생, 프로그래머가 처리할 수 없음, 동적 메모리를 다 사용하거나 스택오버플로우가 발생했을 경우
- 예외 : 프로그램에서 제어할 수 있는 오류, 읽으려는 파일이 없는 경우, 네트웍이나 소켓 연결 오류 등

### Exception 클래스

- 모든 예외 클래스의 최상위 클래스는 Exception 클래스임
- try catch 문으로 예외를 잡아냄
     - try : 예외가 발생할 수 있는 부분
     - catch : 예외가 발생했을 때 실행할 부분
     - finally : 리소스 해제, 예외가 발생했든 안했든 무조건 수행됨, 이 안에 새로운 try catch도 들어올 수 이씀
     - 에러마다 다른 catch문을 사용할 수 있음 

In [1]:
// 프로그램은 어떻게든 돌게 된다는게 중요하다

int[] arr = new int[5];
try {
    for(int i=0;i<=5;i++){
        System.out.println(arr[i]);
    }
}catch(ArrayIndexOutOfBoundsException e){
    System.out.println(e);
    System.out.println("예외처리");
    }
System.out.println("프로그램 종료");    


0
0
0
0
0
java.lang.ArrayIndexOutOfBoundsException: Index 5 out of bounds for length 5
예외처리
프로그램 종료


### try-with-resources
- 리소스를 자동으로 해제할 수 있게
- 리소스가 AutoCloseable을 구현했다면 close()를 명시적으로 호출하지 않아도
- try{}블록에서 오픈된 리소스가 정상적인 경우나 예외가 발생 한경우 모두 자동으로 리소스 해제됨

In [2]:
// AutoCloaseable 구현
// 음 그냥 finally를 클래스 안에다가 구현한거임

public class AutoCloseObj implements AutoCloseable {
    @Override
    public void close() throws Exception{
        System.out.println("Close가 호출되었고 얘는 종료");
    }
}

In [3]:
// 에러가 나든 안나든 close가 호출됨

// try에다가 생성자 인자 넘겨줌
// 자바 9부터는 try안에 변수만 넣어줘도 됨
try(AutoCloseObj obj = new AutoCloseObj()) {
    System.out.println("실행");
    throw new Exception();
} catch(Exception e) {
    System.out.println("에러");
}

실행
Close가 호출되었고 얘는 종료
에러


### 예외처리 미루기
- try 안쓰고 메서드 선언부에 throws를 추가
- 예외가 발생한 메서드에서 예외처리를 하지 않고 이 메소드를 호출한 곳에서 예외 처리한다

In [4]:
public class ThrowsException{
    // 메서드 선언부에 예외
    // 원래는 메서드 안에서 try catch로 예외를 잡아내야했지만, 호출한 곳에서 처리하라는 것
    public Class loadClass(String fileName, String className) throws FileNotFoundException, ClassNotFoundException{
        FileInputStream fis = new FileInputStream(fileName);
        Class c = Class.forName(className);
        return c;
    }
}

In [5]:
ThrowsException test = new ThrowsException();

try {
    test.loadClass("b.txt", "java.lang.string");
} catch (FileNotFoundException e) {
    System.out.println(e);
} catch (ClassNotFoundException e) {
    System.out.println(e);
} catch (Exception e) {
    // 다중예외처리
    // 최상위 클래스 exception : 모든 예외는 이걸로 업캐스팅됨
    // 디폴트 exception : 무조건 맨 아래다가 넣어야함
    System.out.println(e);
}

System.out.println("end");

java.io.FileNotFoundException: b.txt (No such file or directory)
end


### 예외 직접 만들기

In [6]:
// Exception 클래스 상속받은 새로운 예외 클래스

public class IDFormatException extends Exception {
    public IDFormatException(String message) {
        super(message);
    }
}

In [7]:
public class IDFormatTest {
    private String userId;
    
    public String getUserId(){
        return userId;
    }
    
    public void setUserId(String userId)throws IDFormatException{
    
        if (userId == null) {
            throw new IDFormatException("아이디는 눌일수 없지렁");
        } else if(userId.length() < 8 || userId.length() > 20) {
            throw new IDFormatException("아이디는 8자 이상 20자 이하로 쓰세영");
        }
        this.userId = userId;
    }
}

In [8]:
IDFormatTest idTest = new IDFormatTest();

String myId = null;
try {
    idTest.setUserId(myId);
} catch(IDFormatException e) {
    System.out.println(e);
}


REPL.$JShell$21$IDFormatException: 아이디는 눌일수 없지렁


In [9]:
String myId2 = "23125";

try {
    idTest.setUserId(myId2);
} catch(IDFormatException e) {
    System.out.println(e);
}


REPL.$JShell$21$IDFormatException: 아이디는 8자 이상 20자 이하로 쓰세영


## 자바 입출력

### 자바 입출력 스트림
- 다양한 입출력 장치에 독립적으로 일관성있는 입출력 방식 제공
- 입출력이 구현되는 곳에서는 모두 I/O 스트림을 사용
- 입력 스트림 : 대상으로부터 자료를 읽어 들임
- 출력 스트림 : 대상으로 자료를 출력함

### 표준 입출력(Standard IO)
- System 클래스의 표준 입출력 멤버들
- 예외처리 해야댐
```java
public class System {
    public static PrintStream out;
    public static InputStream in;
    public static PrintStream err;
}
```

In [6]:
System.out.println("입력: ");

// 바이트 단위로 읽어 들이는 시스템
// close 할필요 없음
try {
    int i;
    while ((i = System.in.read()) != '\n'){
        System.out.print((char)i);
    }
   
} catch(IOException e) {
    e.printStackTrace();
}


입력: 
ssdfdgdg
ssdfdgdg

In [8]:
// 한글의 경우
// read는 1바이트씩 읽으니까(인풋스트림)
// 2바이트씩 읽으려면 보조스트림이 필요한거임
System.out.println("입력후 끝이라고 쓰셈: ");

// 바이트 단위로 읽어 들이는 시스템
// close 할필요 없음
try {
    int i;
    // isr이 여기서 보조스트림
    InputStreamReader isr = new InputStreamReader(System.in);
    while ((i = isr.read()) != '끝'){
        System.out.print((char)i);
    }
   
} catch(IOException e) {
    e.printStackTrace();
}



입력후 끝이라고 쓰셈: 
안녕나는max라네끝
안녕나는max라네

### Scanner
- java.util 패키지의 입력 클래스
- 문자뿐 아니라 정수 실수등 다양한 자료형을 읽음

### Console
- System.in 말구 콘솔에서 표준 입출력 가능하게 하는 클래스
- 콘솔에서만 들어감

In [10]:
import java.io.Console;

Console console = System.console();
System.out.println("이름: ");
String name = console.readLine();
System.out.println("비번: ");
// 이것도 바이트로 가져오나보넹
char[] password = console.readPassword()

System.out.println(name);
System.out.println(password);

이름: 


EvalException: null